## Nearest-node Map Matching

#### = 各GPS測位点を最寄りのノード（交差点）に割り当てるマッチングアルゴリズム

- 位置情報データ: df

- ある一人のユーザの位置情報データ: df_1user

In [ ]:
import pandas as pd

df = pd.read_csv("位置情報csvのパス") #パスを記入
df_1user = df[df["userid"]=="調べたいユーザのID"] #ユーザIDを記入

In [ ]:
import osmnx as ox
import geopandas as gpd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point

# -----------------------------
# 1) OSMグラフの準備（中区などポリゴンで取得）
# -----------------------------
place = "Naka Ward, Yokohama, Kanagawa, Japan"
gdf_place = ox.geocode_to_gdf(place)
polygon = gdf_place.loc[0, "geometry"]

G = ox.graph_from_polygon(polygon, network_type="all", simplify=True)
G_proj = ox.project_graph(G)
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_proj)

# -----------------------------
# 2) ユーザーGPSデータをGeoDataFrame化
# df_1user: lon, lat, recordedat列を持つ DataFrame
# -----------------------------
# recordedat を datetime 型に変換
df_1user["recordedat"] = pd.to_datetime(df_1user["recordedat"])

gdf_user = gpd.GeoDataFrame(
    df_1user,
    geometry=gpd.points_from_xy(df_1user.lon, df_1user.lat),
    crs="EPSG:4326"
)
gdf_user = gdf_user.to_crs(G_proj.graph['crs'])
gdf_user = gdf_user.sort_values("recordedat").reset_index(drop=True)

KD-treeを用いて最近傍点探索を高速化

In [ ]:
# -----------------------------
# 3) KD-tree で最寄り交差点（次数>=3）を取得
# -----------------------------
deg_series = pd.Series(dict(G_proj.degree()))
nodes_gdf["degree"] = nodes_gdf.index.map(deg_series)

cand_nodes = nodes_gdf[nodes_gdf["degree"] >= 3].copy()
coords = np.vstack((cand_nodes["x"].values, cand_nodes["y"].values)).T

from scipy.spatial import cKDTree
tree = cKDTree(coords)

pt_coords = np.vstack((gdf_user.geometry.x, gdf_user.geometry.y)).T
dists, idxs = tree.query(pt_coords, k=1)

gdf_user["nearest_node"] = cand_nodes.index.values[idxs]
gdf_user["snap_dist_m"] = dists

# -----------------------------
# 4) 経路復元（shortest path）／連続重複除去
# -----------------------------
node_seq = gdf_user["nearest_node"].tolist()
node_seq = [node_seq[i] for i in range(len(node_seq)) if i == 0 or node_seq[i] != node_seq[i-1]]

full_path_nodes = []
for a, b in zip(node_seq[:-1], node_seq[1:]):
    try:
        sp = nx.shortest_path(G_proj, a, b, weight="length")
    except nx.NetworkXNoPath:
        sp = [a, b]
    if full_path_nodes:
        full_path_nodes.extend(sp[1:])
    else:
        full_path_nodes.extend(sp)

# -----------------------------
# 5) 可視化（GPS vs マップマッチング）
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
path_x = [G_proj.nodes[n]['x'] for n in full_path_nodes]
path_y = [G_proj.nodes[n]['y'] for n in full_path_nodes]

fig, ax = plt.subplots(figsize=(10, 10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, color='blue', linewidth=2, alpha=0.7, label='Original GPS')
ax.plot(path_x, path_y, color='red', linewidth=2, alpha=0.7, label='Matched Path')
ax.legend(fontsize=12)
plt.title("Original GPS vs Map-Matched Path (nearest-nodes matching)")
plt.show()

以下は``ox.distance.nearest_nodes``を用いたバージョンの最寄り交差点探索アルゴリズム

*``ox.distance.nearest_edges`` は内部で KD-tree を使っているので速い!!

In [ ]:
# -----------------------------
# 3) 最近傍ノード探索（ox.distance.nearest_nodes を使用）
# -----------------------------
# GPS座標を配列に変換
x = gdf_user.geometry.x.values
y = gdf_user.geometry.y.values

# 最近傍ノードを一括検索
nearest_nodes = ox.distance.nearest_nodes(G_proj, X=x, Y=y, return_dist=True)

# 戻り値は (node_ids, dists) のタプル
gdf_user["nearest_node"] = nearest_nodes[0]
gdf_user["snap_dist_m"] = nearest_nodes[1]

# -----------------------------
# 4) 経路復元（shortest path）／連続重複除去
# -----------------------------
node_seq = gdf_user["nearest_node"].tolist()
# 連続重複を削除
node_seq = [node_seq[i] for i in range(len(node_seq)) if i == 0 or node_seq[i] != node_seq[i-1]]

full_path_nodes = []
for a, b in zip(node_seq[:-1], node_seq[1:]):
    try:
        sp = nx.shortest_path(G_proj, a, b, weight="length")
    except nx.NetworkXNoPath:
        sp = [a, b]
    if full_path_nodes:
        full_path_nodes.extend(sp[1:])  # 重複を避ける
    else:
        full_path_nodes.extend(sp)

# -----------------------------
# 5) 可視化（GPS vs マップマッチング）
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
path_x = [G_proj.nodes[n]['x'] for n in full_path_nodes]
path_y = [G_proj.nodes[n]['y'] for n in full_path_nodes]

fig, ax = plt.subplots(figsize=(10, 10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, color='blue', linewidth=2, alpha=0.7, label='Original GPS')
ax.plot(path_x, path_y, color='red', linewidth=2, alpha=0.7, label='Matched Path')
ax.legend(fontsize=12)
plt.title("Original GPS vs Map-Matched Path (nearest-nodes matching)")
plt.show()